In [1]:
from __future__ import print_function
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
import pandas as pd
import numpy as np
import resnet
import os
from tqdm import tqdm_notebook
from keras.preprocessing.image import load_img
from keras.models import load_model,Model
from keras.layers import (
    Conv2D,
    Input,
    Activation,
    Dense,
    Flatten,BatchNormalization,Add,MaxPooling2D,Dropout,AveragePooling2D
)
from keras import backend as K
import gc
print("Done!")

Using TensorFlow backend.


Done!


In [2]:
train_df=pd.read_csv(os.getcwd()+'/train.csv')
test_df=pd.read_csv(os.getcwd()+'/val.csv')

In [3]:
train_df["data"] = [np.array(load_img(os.getcwd()+"/train/{}".format(idx), target_size=(122,122))) / 255 for idx in tqdm_notebook(train_df['images'])]

A Jupyter Widget

In [4]:
train_df['data'][0].shape

(122, 122, 3)

In [5]:
test_df["data"] = [np.array(load_img(os.getcwd()+"/train/{}".format(idx), target_size=(122,122))) / 255 for idx in tqdm_notebook(test_df['images'])]

A Jupyter Widget

In [6]:
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.1e-6)
early_stopper = EarlyStopping(min_delta=0.001, patience=20)

batch_size = 32
nb_classes = 2
nb_epoch = 200
data_augmentation = False
csv_logger = CSVLogger('resnet18_cat_dog.csv')
img_rows, img_cols = 122, 122
img_channels = 3

In [7]:
y_train=np_utils.to_categorical(train_df['label'], nb_classes)

In [8]:
y_test=np_utils.to_categorical(test_df['label'], nb_classes)

In [9]:
X_train = np.array(train_df['data'].tolist())
X_test = np.array(test_df['data'].tolist())

In [10]:
def BatchActivate(x):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def convolution_block(x, filters, size, strides=(1,1), padding='same', activation=True):
    x = Conv2D(filters, size, strides=strides, padding=padding)(x)
    if activation == True:
        x = BatchActivate(x)
    return x

def residual_block(blockInput, num_filters=16, batch_activate = False):
    x = BatchActivate(blockInput)
    x = convolution_block(x, num_filters, (3,3) )
    x = convolution_block(x, num_filters, (3,3), activation=False)
    x = Add()([x, blockInput])
    if batch_activate:
        x = BatchActivate(x)
    return x

In [11]:
#model = resnet.ResnetBuilder.build_resnet_50((img_channels, img_rows, img_cols), nb_classes)

input_layer=Input((122,122,3))

conv1 = Conv2D(64, (7,7), strides=(2,2) ,activation='relu', padding="same")(input_layer)
p0 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(conv1)

conv1 = Conv2D(64, (3, 3), activation='relu', padding="same")(p0)
conv1 = Conv2D(64, (3, 3), activation=None, padding="same")(conv1)
conv1 = residual_block(conv1,64)
conv1 = residual_block(conv1,64,True)
pool1 = Dropout(0.1)(conv1)

conv1 = Conv2D(64, (3, 3), activation='relu', padding="same")(pool1)
conv1 = Conv2D(64, (3, 3), activation=None, padding="same")(conv1)
conv1 = residual_block(conv1,64)
conv1 = residual_block(conv1,64,True)
pool1 = Dropout(0.1)(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding="same")(pool1)
conv2 = Conv2D(128, (3, 3), activation=None, padding="same")(conv2)
conv2 = residual_block(conv2,128)
conv2 = residual_block(conv2,128)
pool2 = Dropout(0.1)(conv2)
pool2 = MaxPooling2D()(pool2)

conv2 = Conv2D(64, (3, 3), activation='relu', padding="same")(pool2)
conv2 = Conv2D(128, (3, 3), activation=None, padding="same")(pool2)
conv2 = residual_block(conv2,128)
conv2 = residual_block(conv2,128,True)
pool2 = Dropout(0.1)(conv2)

conv3 = Conv2D(64, (3, 3), activation='relu', padding="same")(pool2)
conv3 = Conv2D(256, (3, 3), activation=None, padding="same")(conv3)
conv3 = residual_block(conv3,256)
conv3 = residual_block(conv3,256,True)
pool3 = Dropout(0.1)(conv3)

conv3 = Conv2D(256, (3, 3), activation='relu', padding="same")(pool3)
conv3 = Conv2D(256, (3, 3), activation=None, padding="same")(conv3)
conv3 = residual_block(conv3,256)
conv3 = residual_block(conv3,256,True)
pool3 = Dropout(0.1)(conv3)

conv3 = Conv2D(256, (3, 3), activation='relu', padding="same")(pool3)
conv3 = Conv2D(512, (3, 3), activation=None, padding="same")(conv3)
conv3 = residual_block(conv3,512)
conv3 = residual_block(conv3,512,True)
conv3 = Conv2D(256, (3, 3), activation='relu', padding="same")(conv3)

if K.image_dim_ordering() == 'tf':
    ROW_AXIS = 1
    COL_AXIS = 2
    CHANNEL_AXIS = 3
else:
    CHANNEL_AXIS = 1
    ROW_AXIS = 2
    COL_AXIS = 3

block_shape = K.int_shape(conv3)
pool4 = AveragePooling2D(pool_size=(block_shape[ROW_AXIS], block_shape[COL_AXIS]),
                         strides=(1, 1))(conv3)


fla = Flatten()(pool4)
dense = Dense(units=2, kernel_initializer="he_normal",
                      activation="softmax")(fla)
model=Model(inputs=input_layer,output=dense)


/home/wenjie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:68: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [12]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [13]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 122, 122, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 61, 61, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 30, 30, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 30, 30, 64)   36928       max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
conv2d_3 (

In [14]:
model_checkpoint = ModelCheckpoint("./Third_net.model", save_best_only=True, verbose=1)
model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=nb_epoch,
              validation_data=(X_test, y_test),
              shuffle=True,
              callbacks=[lr_reducer, early_stopper, csv_logger,model_checkpoint])


Train on 8000 samples, validate on 2000 samples
Epoch 1/200
8000/8000 [==============================] - 56s 7ms/step - loss: 0.7151 - acc: 0.4958 - val_loss: 0.6933 - val_acc: 0.4945

Epoch 00001: val_loss improved from inf to 0.69327, saving model to ./Third_net.model
Epoch 2/200
8000/8000 [==============================] - 50s 6ms/step - loss: 0.6892 - acc: 0.5341 - val_loss: 5.4822 - val_acc: 0.5065

Epoch 00002: val_loss did not improve from 0.69327
Epoch 3/200
8000/8000 [==============================] - 50s 6ms/step - loss: 0.6873 - acc: 0.5406 - val_loss: 0.6788 - val_acc: 0.5550

Epoch 00003: val_loss improved from 0.69327 to 0.67876, saving model to ./Third_net.model
Epoch 4/200
8000/8000 [==============================] - 51s 6ms/step - loss: 0.6800 - acc: 0.5736 - val_loss: 0.6888 - val_acc: 0.5220

Epoch 00004: val_loss did not improve from 0.67876
Epoch 5/200
8000/8000 [==============================] - 51s 6ms/step - loss: 0.6765 - acc: 0.5715 - val_loss: 3.2371 - val_ac

8000/8000 [==============================] - 52s 6ms/step - loss: 0.0489 - acc: 0.9814 - val_loss: 0.7523 - val_acc: 0.8080

Epoch 00042: val_loss did not improve from 0.43999
Epoch 43/200
8000/8000 [==============================] - 52s 6ms/step - loss: 0.0298 - acc: 0.9900 - val_loss: 0.6153 - val_acc: 0.8505

Epoch 00043: val_loss did not improve from 0.43999
Epoch 44/200
8000/8000 [==============================] - 52s 6ms/step - loss: 0.0255 - acc: 0.9908 - val_loss: 0.6925 - val_acc: 0.8450

Epoch 00044: val_loss did not improve from 0.43999
Epoch 45/200
8000/8000 [==============================] - 52s 6ms/step - loss: 0.0200 - acc: 0.9934 - val_loss: 0.6738 - val_acc: 0.8555

Epoch 00045: val_loss did not improve from 0.43999
Epoch 46/200
8000/8000 [==============================] - 52s 6ms/step - loss: 0.0164 - acc: 0.9954 - val_loss: 0.7238 - val_acc: 0.8505

Epoch 00046: val_loss did not improve from 0.43999
Epoch 47/200
8000/8000 [==============================] - 52s 6ms/st

In [15]:
#model=load_model("./resnet_cat_dog.model")

In [16]:
#model=load_model("./Net2.model")

In [17]:
#model=load_model("./Net3.model")

In [18]:
#model=load_model("./Res_50.model")

In [19]:
del X_train,X_test
gc.collect()

0

In [20]:
test_y=pd.read_csv(os.getcwd()+'/test.csv')
test=pd.DataFrame()
test['label']=test_y['label']
test['images']=test_y['images']


        

In [21]:
print(len(test['images']))

8000


In [22]:
test["data"] = [np.array(load_img(os.getcwd()+"/train/{}".format(idx), target_size=(122,122))) / 255 for idx in tqdm_notebook(test['images'])]

A Jupyter Widget

In [23]:
test_in=np.array(test['data'].tolist())
test_re=np.array(test['label'].tolist())

In [24]:
test_re[0:10]

array([1, 0, 0, 1, 0, 0, 1, 1, 1, 0])

In [25]:

pre=model.predict(test_in)

In [26]:
pre[0:10]

array([[1.60368960e-04, 9.99839664e-01],
       [9.99997735e-01, 2.28557928e-06],
       [1.00000000e+00, 1.07541524e-10],
       [6.92855508e-07, 9.99999285e-01],
       [6.17872807e-04, 9.99382138e-01],
       [1.00000000e+00, 3.84217940e-13],
       [4.80419430e-07, 9.99999523e-01],
       [1.39408976e-05, 9.99986053e-01],
       [7.55863249e-01, 2.44136721e-01],
       [3.74202020e-02, 9.62579846e-01]], dtype=float32)

In [27]:
pre=np.argmax(pre,axis=1)   


In [28]:
test_re=np.array(test_re)

In [29]:
print(pre[0:10])
print(test_re[0:10])

[1 0 0 1 1 0 1 1 0 1]
[1 0 0 1 0 0 1 1 1 0]


In [30]:
del(test)
gc.collect()

14

In [31]:
correct=0
wrong=0
for i in range(len(pre)):
    if pre[i]==test_re[i]:
        correct+=1
    else:
        wrong+=1
print("Correct:",correct)
print("Wrong:",wrong)
print("Acc:",float(correct/(correct+wrong)))

Correct: 6874
Wrong: 1126
Acc: 0.85925
